<a href="https://colab.research.google.com/github/julvc/python_diplo/blob/master/MP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Miniproyecto 2: Machine Learning con Valor
#### **Curso:** Introducción a Minería de Datos y Machine Learning

---

**¡Felicidades por tu excelente trabajo en el MP1!**

Tu labor en el preprocesamiento de la base de datos ha causado un gran impacto en **AMAZOFF**.

El equipo de Customer Experience ha quedado impresionado con tu habilidad para transformar datos sucios en información
útil y ahora quieren aprovechar tus talentos para enfrentar un nuevo desafío.

Nos han pedido desarrollar un **Predictor de Ratings de Pedidos** utilizando los datos preprocesados de ratings de
pedidos de nuestra plataforma de e-commerce.

# Pauta de Evaluación

Este MP2 está enfocado en responder preguntas de negocio relevantes para **AMAZOFF**.

### Preguntas de Negocio

1. (15 puntos) El equipo necesita extraer la máxima información de los datos. ¿Cómo modificarías la base de datos para ser usada con modelos de ML?
2. (15 puntos) ¡Wow! Gran *feature engineering*. Toca armar tu set para entrenar el modelo. ¿Podrías generar gráficas para el equipo de Customer Experience que expliquen mejor los datos nuevos? ¡No olvides explicarlas!
3. (10 puntos) Dicen que se pueden encontrar patrones en tus datos de entrenamiento. ¿Qué patrones encuentras en los pedidos? ¿Cómo pueden ser de utilidad para mejorar las ventas de AMAZOFF?
4. (25 puntos) ¡Viva el ML! Toca predecir la satisfacción del cliente. ¿Es posible predecir bien el `rating_class`?
5. (20 puntos) El equipo de Customer Experience dice que un solo modelo no es suficiente. ¿Podrías implementar otro?
6. (15 puntos) ¡Dos modelos! Eso es genial. ¿Qué modelo funcionó peor? ¿Si utilizas PCA podría mejor?

### Tener en consideración:

En caso de que el código esté bien, pero no se responda (usando celdas de texto) la pregunta de negocio (sección **Explicación**), **se asignará máximo la mitad de puntos** de esa pregunta.

El miniproyecto está diseñado para completar código. Suba el notebook solo con el código para responder la pregunta de negocio.

> # ¡NO OLVIDES GUARDAR Y SUBIR EL NOTEBOOK A LA PLATAFORMA CUANDO TERMINES! FORMATO .ipynb

# Preámbulo

**¡Cuidado!** En esta sección solo puedes modificar la sección de PÁRAMETROS.

In [1]:
# ¡No modificar esta celda! No está permitido usar librerías adicionales.

# Association Rules
from mlxtend.frequent_patterns import apriori, association_rules

# Machine Learning
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA

# General Data Science
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [4]:
# Esta configuración permite que los dataframes se muestren completos.
import warnings ; warnings.warn = lambda *args,**kwargs: None
pd.set_option('display.max_columns', None)

In [5]:
# PARÁMETROS (puedes modificar esta sección)

DATA_PATH = 'ecommerce.csv' # Si tienes el archivo en otro lugar, cámbialo.
RANDOM_STATE = 0 # ¡Cámbialo a tu número favorito!

In [6]:
# ¡No modificar! Esta celda se encarga de cargar los datos.
df = pd.read_csv(DATA_PATH)
df.purchased = pd.to_datetime(df.purchased)
df.delivered = pd.to_datetime(df.delivered)
df.estimated_delivery = pd.to_datetime(df.estimated_delivery)
print(f'{df.shape[0]} rows x {df.shape[1]} columns')
df.head(5)

63606 rows x 14 columns


,order,product,seller,customer,city,state,category,price,shipping,payment,purchased,delivered,estimated_delivery,rating
0,0,0,0,0,unknown,SP,health,223.51,84.65,credit card,2017-10-22 18:57:54,2017-10-26 22:19:52,2017-11-09 12:00:00,1.0
1,1,1,1,1,unknown,RJ,health,170.80,23.79,credit card,2018-06-20 21:40:31,2018-07-03 22:51:22,2018-07-24 12:00:00,4.5
2,2,2,2,2,unknown,SP,technology,64.40,17.38,credit card,2018-02-16 16:19:31,2018-02-27 01:29:50,2018-03-08 12:00:00,3.5
3,3,1,3,3,uberlandia,MG,technology,264.50,30.72,credit card,2018-08-18 18:04:29,2018-08-27 20:03:51,2018-09-19 12:00:00,3.0
4,4,3,4,4,unknown,SP,technology,779.90,30.66,credit card,2017-12-22 16:44:04,2018-01-05 19:22:49,2018-01-18 12:00:00,2.5


In [7]:
# ¿Cuántos valores únicos hay en cada columna?
for col in df.columns:
    print(f'{col}: {df[col].nunique()} unique values')

order: 63606 unique values
product: 18 unique values
seller: 25 unique values
customer: 63606 unique values
city: 26 unique values
state: 5 unique values
category: 5 unique values
price: 4433 unique values
shipping: 4849 unique values
payment: 2 unique values
purchased: 53475 unique values
delivered: 53496 unique values
estimated_delivery: 433 unique values
rating: 9 unique values


# 1. El equipo necesita extraer la máxima información de los datos. ¿Cómo modificarías la base de datos para ser usada con modelos de ML?

⁉️ **Consideración:** La última celda de esta sección es la de cálculo de `rating_class`, mantener el orden. Agregar las celdas para modificar
los datos, antes de esta. No olvides modificar el código para calcular la clase de rating.

ℹ️ **HINT:** Utiliza una celda para cada columna. ¿Qué columnas no sirven? ¿Qué columnas necesito que sean números? ¿Nuevas columnas?

📖 **Referencia Externa:** [Feature Engineering](https://www.freecodecamp.org/news/feature-engineering-techniques-for-structured-data/).

---

✅ **Explicación:**

[ AGREGA TU RESPUESTA ACÁ ]

In [33]:
#df.value_counts('city', normalize=True).head(5) unknown,sao paulo, rio de janeiro
#df.value_counts('state', normalize=True).head(5) SP, RJ, MG
#df.value_counts('category', normalize=True)  fashion, technology, toys, health, beauty
#df.value_counts('payment', normalize=True).head(5) credit card, other

,proportion
product,
0,0.099409
9,0.098497
1,0.097931
6,0.084913
4,0.084882


In [1]:
# [ COLUMNAS PARA CITY Y STATE]
top_cities = df['city'].value_counts().head(3).index.to_list()
#df['is_top_product'] = df['product'].isin(top_products).astype(int)

NameError: name 'df' is not defined

In [ ]:
# [ AGREGA TU CÓDIGO AQUÍ ] ¿Necesitas agregar más celdas? Adelante.

In [ ]:
# [ AGREGA TU CÓDIGO AQUÍ ] ¿Necesitas agregar más celdas? Adelante. RATING_CLASS

Debido a que estaremos realizando clasificación, pasaremos `rating`de numérico a categórico. Deberás modificar `RATING_BINS`
y `RATING_LABELS` para seleccionar cómo realizar el *binning* de esta clase.

**Ejemplo:**
```python
    RATING_BINS = [0, 2, 3.5, 5]
    RATING_LABELS = ['low', 'medium', 'high']
```

In [ ]:
# Modifica RATING_BINS y RATING_LABELS según consideres.
RATING_BINS = []
RATING_LABELS = []

################################################################################

# ¡No modificar! Este código se encarga de crear la columna rating_class.
df['rating_class'] = pd.cut(df['rating'], bins=RATING_BINS, labels=RATING_LABELS, include_lowest=True)
df.value_counts(['rating', 'rating_class'], normalize=True, sort=False, dropna=False)

**¡Ten cuidado!** Debes intentar que tus `rating_class` no sean demasiado imbalancedas. Por ejemplo, que el 90% de los datos
sean *low*, 5% *medium* y 5% *high*. Es preferible algo como: 40% *low*, 30% *medium* y 30% *high*.

In [ ]:
# ¡No modificar!
df.value_counts('rating_class', normalize=True, sort=False, dropna=False)

# 2. ¡Wow! Gran *feature engineering*. Toca armar tu set para entrenar el modelo. ¿Podrías generar gráficas para el equipo de Customer Experience que expliquen mejor los datos nuevos? ¡No olvides explicarlas!

⁉️ **Consideración:** Modifica la lista de columnas a eliminar para el *train*. Debes de hacer por lo menos dos gráficos.

ℹ️ **HINT 1:** ¿Por qué tus gráficas aportan al equipo de Customer Experience? ¡Explica tus nuevas columnas! ¿No tienes? 😓

ℹ️ **HINT 2:** Un buen gráfico tiene título, *labels*, leyenda... ¡y mucho más!

📖 **Referencia Externa:** [Tutorial de Seaborn](https://www.datacamp.com/tutorial/seaborn-python-tutorial).

---

✅ **Explicación:**

- **Gráfico 1:** [ AGREGA TU EXPLICACIÓN ACÁ ]

- **Gráfico 2:** [ AGREGA TU EXPLICACIÓN ACÁ ]

In [ ]:
# Gráfico 1: [ TÍTULO DEL GRÁFICO ]

########################################################################################################################

# [ AGREGA TU CÓDIGO AQUÍ ]

In [ ]:
# Gráfico 2: [ TÍTULO DEL GRÁFICO ]

########################################################################################################################

# [ AGREGA TU CÓDIGO AQUÍ ]

In [ ]:
# Modifica la lista de columnas a eliminar según consideres.

x = df.drop(
    columns=[
        # Agrega las columnas que consideres
        # [ AGREGA TU CÓDIGO AQUÍ ]
        'rating', 'rating_class'
    ]
)

########################################################################################################################

# ¡No modificar! Esta celda se encarga de dividir los datos en conjuntos de entrenamiento, validación y prueba.

y = df['rating_class']

x_train_val, x_test, y_train_val, y_test = train_test_split(x, y, test_size=0.2, shuffle=True, random_state=RANDOM_STATE)
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=0.2, shuffle=True, random_state=RANDOM_STATE)

print(f'Train: {x_train.shape[0]} rows')
print(f'Validation: {x_val.shape[0]} rows')
print(f'Test: {x_test.shape[0]} rows')

display(x_train.head(5))

# 3. Dicen que se pueden encontrar patrones en tus datos de entrenamiento. ¿Qué patrones encuentras en los pedidos? ¿Cómo pueden ser de utilidad para mejorar las ventas de AMAZOFF?

⁉️ **Consideración:** Utiliza reglas de asociación para encontrar reglas útiles para nuevas estrategias de venta. Explica los parámetros y métricas de tu modelo.

ℹ️ **HINT 1:** ¿Qué librería se importó en el inicio? Cuidado, ¡solo datos binarios!

ℹ️ **HINT 2:** De las principales reglas encontradas... ¿cómo implementarías una estrategia de venta/marketing utilizando esta información?

📖 **Referencia Externa:** [Documentación de mlxtend](https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/).

---

✅ **Explicación:**

[ AGREGA TU EXPLICACIÓN ACÁ ]

In [ ]:
binary_x_train = x_train.drop(
    columns=[
        # Agrega las columnas que consideres
        # [ AGREGA TU CÓDIGO AQUÍ ]
    ]
).astype(bool)

frequent_itemsets = # [ AGREGA TU CÓDIGO AQUÍ ]
frequent_itemsets

In [ ]:
rules = # [ AGREGA TU CÓDIGO AQUÍ ]
rules

# 4. ¡Viva el ML! Toca predecir la satisfacción del cliente. ¿Es posible predecir bien el `rating_class`?

⁉️ **Consideraciones:**

- Elige tres sets de hiperparámetros **diferentes** para los tres modelos de clasificación de *Random Forest*.
- Debes modificar al menos 4 hiperparámetros. No cuentes `random_state`ni `n_jobs`.
- Tus sets de hiperparámetros deben pasar la prueba de *overfitting* y *underfitting*.
- Utiliza los resultados de la validación para elegir tus hiperparámetros, no los del testeo.
- No olvides interpretar tus resultados e intenta relacionarlos a los intereses de **AMAZOFF** si es posible.

ℹ️ **HINT:** El ML es un proceso iterativo, a veces si es que no funcionan bien tus resultados debes de volver a empezar.

📖 **Referencia Externa:** [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html).

---

✅ **Explicación:**

- **Justifica tus hiperpámetros:** [ AGREGA TU EXPLICACIÓN ACÁ ]
- **¿Hay *overfitting* o *underfitting*:** [ AGREGA TU EXPLICACIÓN ACÁ ]
- **Resultados de Validación:** [ AGREGA TU EXPLICACIÓN ACÁ ]
- **Resultados de Testeo:** [ AGREGA TU EXPLICACIÓN ACÁ ]
- **¿Es posible predecir bien el `rating_class`:** [ AGREGA TU EXPLICACIÓN ACÁ ]

In [ ]:
%%time

# Debes de modificar al menos 4 parámetros, sin contar random_state y n_jobs.

rf1 = RandomForestClassifier(
    # [ AGREGA TU CÓDIGO AQUÍ ]
    random_state=RANDOM_STATE,
    n_jobs=-1
)

rf1.fit(x_train, y_train)

In [ ]:
%%time

# Debes de modificar al menos 4 parámetros, sin contar random_state y n_jobs.

rf2 = RandomForestClassifier(
    # [ AGREGA TU CÓDIGO AQUÍ ]
    random_state=RANDOM_STATE,
    n_jobs=-1
)

rf2.fit(x_train, y_train)

In [ ]:
%%time

# Debes de modificar al menos 4 parámetros, sin contar random_state y n_jobs.

rf3 = RandomForestClassifier(
    # [ AGREGA TU CÓDIGO AQUÍ ]
    random_state=RANDOM_STATE,
    n_jobs=-1
)

rf3.fit(x_train, y_train)

In [ ]:
# ¡No modificar! Esta celda se encarga de evaluar los modelos y mostrar los resultados.

y_val_pred1 = rf1.predict(x_val)
y_val_pred2 = rf2.predict(x_val)
y_val_pred3 = rf3.predict(x_val)

print('[Random Forest] Check Overfitting with Accuracy')
print(f'RF1: train={accuracy_score(y_train, rf1.predict(x_train)):.4f} val={accuracy_score(y_val, y_val_pred1):.4f}')
print(f'RF2: train={accuracy_score(y_train, rf2.predict(x_train)):.4f} val={accuracy_score(y_val, y_val_pred2):.4f}')
print(f'RF3: train={accuracy_score(y_train, rf3.predict(x_train)):.4f} val={accuracy_score(y_val, y_val_pred3):.4f}')

In [ ]:
# ¡No modificar! Esta celda se encarga de mostrar los resultados en el set de validación.

print('[Random Forest] Validation Classification Report')
print('#'*80)
print('RF1')
print(classification_report(y_val, y_val_pred1))
print('#'*80)
print('RF2')
print(classification_report(y_val, y_val_pred2))
print('#'*80)
print('RF3')
print(classification_report(y_val, y_val_pred3))

In [ ]:
# ¡Completa el código! Esta celda se encarga de evaluar los modelos en el set de testeo y mostrar los resultados.

y_pred_rf1 = # [ AGREGA TU CÓDIGO AQUÍ ]
y_pred_rf2 = # [ AGREGA TU CÓDIGO AQUÍ ]
y_pred_rf3 = # [ AGREGA TU CÓDIGO AQUÍ ]

print('[Random Forest] Test Classification Report')
print('#'*80)
print('RF1')
# [ AGREGA TU CÓDIGO AQUÍ ])
print('#'*80)
print('RF2')
# [ AGREGA TU CÓDIGO AQUÍ ])
print('#'*80)
print('RF3')
# [ AGREGA TU CÓDIGO AQUÍ ])

# 5. El equipo de Customer Experience dice que un solo modelo no es suficiente. ¿Podrías implementar otro?

⁉️ **Consideración:** Utiliza otro modelo visto en clase, como *KNN* y repite todos los pasos de la pregunta 4. No utilices *Random Forest*.

ℹ️ **HINT 1:** No olvides elegir 3 set de hiperparámetros y analizar los resultados de validación y testeo.

ℹ️ **HINT 2:** Si utilizas un modelo no visto en clases, debes de explicar su funcionamiento para utilizarlo.

📖 **Referencia Externa:** [KNN](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html).

---

✅ **Explicación:**

- **Justifica tus hiperpámetros:** [ AGREGA TU EXPLICACIÓN ACÁ ]
- **¿Hay *overfitting* o *underfitting*:** [ AGREGA TU EXPLICACIÓN ACÁ ]
- **Resultados de Validación:** [ AGREGA TU EXPLICACIÓN ACÁ ]
- **Resultados de Testeo:** [ AGREGA TU EXPLICACIÓN ACÁ ]
- **¿Es posible predecir bien el `rating_class`:** [ AGREGA TU EXPLICACIÓN ACÁ ]

In [ ]:
# Si usas un modelo no visto en clase: Descomenta e importa tu nuevo modelo de clasificación aquí.
# ¡No olvides agregar a tu explicación cómo funciona el modelo no visto en clase!

# from sklearn.

In [ ]:
# Instancia tu modelo 1 aquí.

# [ AGREGA TU CÓDIGO AQUÍ ]

In [ ]:
# Instancia tu modelo 2 aquí.

# [ AGREGA TU CÓDIGO AQUÍ ]

In [ ]:
# Instancia tu modelo 3 aquí.

# [ AGREGA TU CÓDIGO AQUÍ ]

In [ ]:
# Revisa si hay overfitting (resultado de entrenamiento muy superior al de validación).
# Utiliza: `accuracy_score`.

# [ AGREGA TU CÓDIGO AQUÍ ]

In [ ]:
# Imprime los resultados de tus modelos en el set de validación (classification_report).

# [ AGREGA TU CÓDIGO AQUÍ ]

In [ ]:
# Imprime los resultados de tus modelos en el set de testeo (classification_report).

# [ AGREGA TU CÓDIGO AQUÍ ]

# 6. ¡Dos modelos! Eso es genial. ¿Qué modelo funcionó peor? ¿Si utilizas PCA podría mejor?

⁉️ **Consideración:** Elige la combinación de modelo e hiperparámetros que peor se desempeño entre las preguntas 4 y 5. Aplica PCA con un número de componentes adecuado y corre los resultados otra vez.

ℹ️ **HINT:** ¿Cómo se elige el número de componentes? ¿Y ese gráfico para qué sirve?

📖 **Referencia Externa:** [PCA](https://www.baeldung.com/cs/pca).

---

✅ **Explicación:**

- **¿Por qué elegiste ese número de componentes?** [ AGREGA TU EXPLICACIÓN ACÁ ]
- **Resultados de Validación:** [ AGREGA TU EXPLICACIÓN ACÁ ]
- **Resultados de Testeo:** [ AGREGA TU EXPLICACIÓN ACÁ ]
- **¿Mejoraron los resultados al usar PCA? ¿Por qué?** [ AGREGA TU EXPLICACIÓN ACÁ ]

In [ ]:
# ¡No modificar! Gráfico de Varianza Explicada por Número de Componentes

explained_variance = []
for i in range(1, 8):
    pca = PCA(n_components=i)
    pca.fit(x_train)
    explained_variance.append(pca.explained_variance_ratio_.sum())

plt.plot(range(1, 8), explained_variance, marker='o')
plt.xlabel('Number of Components')
plt.ylabel('Explained Variance')
plt.title('PCA Explained Variance')
plt.show()

In [ ]:
# Elige el número de componentes que consideres adecuado.

pca = PCA(# [ AGREGA TU CÓDIGO AQUÍ ])

########################################################################################################################

# ¡No modificar! Esta celda se encarga de transformar los datos con PCA.

pca.fit(x_train)

x_train_pca = pca.transform(x_train)
x_val_pca = pca.transform(x_val)
x_test_pca = pca.transform(x_test)

In [ ]:
# Elige el modelo que deseas utilizar. Selecciona los hiperparámetros que consideres. ¿Elegiste la peor combinación anterior?
#     Ejemplo 1: model = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE).
#     Ejemplo 2: model = KNeighborsClassifier(n_neighbors=10).

model = # [ AGREGA TU CÓDIGO AQUÍ ]

########################################################################################################################

# ¡No modificar! Este código se encarga de entrenar el modelo y mostrar los resultados en el set de validación.

model.fit(x_train_pca, y_train)

y_val_pca_pred = model.predict(x_val_pca)

print(f'[{model.__class__.__name__}] Validation Classification Report')
print(classification_report(y_val, y_val_pca_pred))

In [ ]:
# ¡No modificar! Esta celda se encarga de mostrar los resultados en el set de test.

y_test_pca_pred = model.predict(x_test_pca)

print(f'[{model.__class__.__name__}] Test Classification Report')
print(classification_report(y_test, y_test_pca_pred))

> # ¡NO OLVIDES GUARDAR Y SUBIR EL NOTEBOOK A LA PLATAFORMA CUANDO TERMINES! FORMATO .ipynb